# 📊 Analyse des Données d'Emploi Adzuna

Ce notebook charge les données JSON d'Adzuna et les transforme en DataFrame pandas pour l'analyse.


In [1]:
import pandas as pd
import numpy as np
import json
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

print("✅ Librairies importées avec succès !")


✅ Librairies importées avec succès !


## 📂 Chargement des données JSON


In [7]:
# Charger le fichier JSON
with open('../data/jobs_data.json', 'r', encoding='utf-8') as f:
    data = json.load(f)

# Afficher les métadonnées
print("📊 Métadonnées du dataset:")
for key, value in data['metadata'].items():
    print(f"  {key}: {value}")

print(f"\n📋 Nombre total d'offres dans le JSON: {len(data['jobs'])}")

# Aperçu de la première offre pour comprendre la structure
print(f"\n👀 Structure d'une offre d'emploi:")
if data['jobs']:
    first_job = data['jobs'][0]
    for key, value in first_job.items():
        if isinstance(value, dict):
            print(f"  {key}: {type(value)} - {list(value.keys())}")
        elif isinstance(value, list):
            print(f"  {key}: {type(value)} - {len(value)} éléments")
        else:
            print(f"  {key}: {type(value)} - {str(value)[:50]}{'...' if len(str(value)) > 50 else ''}")


📊 Métadonnées du dataset:
  search_term: data
  total_jobs: 35000
  scraping_date: 2025-06-05T18:58:08.096210
  api_source: Adzuna

📋 Nombre total d'offres dans le JSON: 35000

👀 Structure d'une offre d'emploi:
  contract_time: <class 'str'> - full_time
  created: <class 'str'> - 2025-06-05T18:08:34Z
  __CLASS__: <class 'str'> - Adzuna::API::Response::Job
  title: <class 'str'> - Mission mentorat - Data Engineer
  salary_is_predicted: <class 'str'> - 0
  longitude: <class 'float'> - 2.3365
  location: <class 'dict'> - ['display_name', '__CLASS__', 'area']
  contract_type: <class 'str'> - contract
  salary_min: <class 'int'> - 108000
  id: <class 'str'> - 5234723007
  salary_max: <class 'int'> - 192000
  redirect_url: <class 'str'> - https://www.adzuna.fr/land/ad/5234723007?se=JmU6xS...
  adref: <class 'str'> - eyJhbGciOiJIUzI1NiJ9.eyJzIjoiSm1VNnhTeEM4QkdrVkEtY...
  description: <class 'str'> - Description du poste Quelles sont les missions des...
  category: <class 'dict'> - ['__CLASS_

## 🔄 Transformation en DataFrame pandas


In [8]:
def flatten_job_data(job):
    """
    Transforme un objet job complexe en dictionnaire plat pour pandas
    """
    flattened = {}
    
    # Champs simples
    simple_fields = ['id', 'title', 'description', 'created', 'contract_type', 
                    'contract_time', 'salary_min', 'salary_max', 'salary_is_predicted',
                    'latitude', 'longitude', 'redirect_url']
    
    for field in simple_fields:
        flattened[field] = job.get(field, None)
    
    # Champs complexes - Company
    company = job.get('company', {})
    flattened['company_name'] = company.get('display_name', None)
    
    # Champs complexes - Location
    location = job.get('location', {})
    flattened['location_display'] = location.get('display_name', None)
    
    area = location.get('area', [])
    flattened['country'] = area[0] if len(area) > 0 else None
    flattened['region'] = area[1] if len(area) > 1 else None
    flattened['department'] = area[2] if len(area) > 2 else None
    flattened['city'] = area[-1] if len(area) > 0 else None
    
    # Champs complexes - Category
    category = job.get('category', {})
    flattened['category_label'] = category.get('label', None)
    flattened['category_tag'] = category.get('tag', None)
    
    return flattened

# Transformer toutes les offres
jobs_flattened = [flatten_job_data(job) for job in data['jobs']]

# Créer le DataFrame
df_jobs = pd.DataFrame(jobs_flattened)

print(f"✅ DataFrame créé avec {len(df_jobs)} lignes et {len(df_jobs.columns)} colonnes")
print(f"\n📋 Colonnes disponibles:")
for i, col in enumerate(df_jobs.columns, 1):
    print(f"  {i:2d}. {col}")


✅ DataFrame créé avec 35000 lignes et 20 colonnes

📋 Colonnes disponibles:
   1. id
   2. title
   3. description
   4. created
   5. contract_type
   6. contract_time
   7. salary_min
   8. salary_max
   9. salary_is_predicted
  10. latitude
  11. longitude
  12. redirect_url
  13. company_name
  14. location_display
  15. country
  16. region
  17. department
  18. city
  19. category_label
  20. category_tag


## 🧹 Nettoyage et enrichissement des données


In [9]:
# Conversion des types de données
df_jobs['created'] = pd.to_datetime(df_jobs['created'])
df_jobs['salary_min'] = pd.to_numeric(df_jobs['salary_min'], errors='coerce')
df_jobs['salary_max'] = pd.to_numeric(df_jobs['salary_max'], errors='coerce')
df_jobs['latitude'] = pd.to_numeric(df_jobs['latitude'], errors='coerce')
df_jobs['longitude'] = pd.to_numeric(df_jobs['longitude'], errors='coerce')

# Calcul de salaires moyens et conversion en k€
df_jobs['salary_avg'] = (df_jobs['salary_min'] + df_jobs['salary_max']) / 2
df_jobs['salary_min_k'] = df_jobs['salary_min'] / 1000
df_jobs['salary_max_k'] = df_jobs['salary_max'] / 1000
df_jobs['salary_avg_k'] = df_jobs['salary_avg'] / 1000

# Nettoyage des titres et descriptions
df_jobs['title_clean'] = df_jobs['title'].str.strip()
df_jobs['description_length'] = df_jobs['description'].str.len()

# Extraction des informations géographiques
df_jobs['is_paris'] = df_jobs['location_display'].str.contains('Paris', case=False, na=False)
df_jobs['is_ile_de_france'] = df_jobs['region'].str.contains('Ile-de-France', case=False, na=False)

# Identification des types de postes
df_jobs['is_data_scientist'] = df_jobs['title_clean'].str.contains('Data Scientist', case=False, na=False)
df_jobs['is_data_analyst'] = df_jobs['title_clean'].str.contains('Data Analyst', case=False, na=False)
df_jobs['is_data_engineer'] = df_jobs['title_clean'].str.contains('Data Engineer', case=False, na=False)
df_jobs['is_alternance'] = df_jobs['title_clean'].str.contains('alternance', case=False, na=False)

print("✅ Nettoyage et enrichissement terminés !")
print(f"📊 Nouvelles colonnes ajoutées: {len(df_jobs.columns) - len(jobs_flattened[0])}")
print(f"📈 DataFrame final: {len(df_jobs)} lignes, {len(df_jobs.columns)} colonnes")


✅ Nettoyage et enrichissement terminés !
📊 Nouvelles colonnes ajoutées: 12
📈 DataFrame final: 35000 lignes, 32 colonnes


## 👀 Aperçu du DataFrame


In [10]:
# Afficher les premières lignes
print("🔍 Aperçu des premières lignes du DataFrame:")
display(df_jobs.head())

# Informations générales
print(f"\n📊 Informations générales:")
print(f"  📋 Nombre d'offres: {len(df_jobs)}")
print(f"  📅 Période: {df_jobs['created'].min().strftime('%Y-%m-%d')} à {df_jobs['created'].max().strftime('%Y-%m-%d')}")
print(f"  🏢 Entreprises uniques: {df_jobs['company_name'].nunique()}")
print(f"  📍 Lieux uniques: {df_jobs['location_display'].nunique()}")

# Informations sur les salaires
salary_data = df_jobs.dropna(subset=['salary_avg_k'])
if len(salary_data) > 0:
    print(f"\n💰 Informations salariales:")
    print(f"  📊 Offres avec salaire: {len(salary_data)} ({len(salary_data)/len(df_jobs)*100:.1f}%)")
    print(f"  💵 Salaire moyen: {salary_data['salary_avg_k'].mean():.1f}k€")
    print(f"  📈 Salaire médian: {salary_data['salary_avg_k'].median():.1f}k€")
    print(f"  ⬇️ Salaire min: {salary_data['salary_avg_k'].min():.1f}k€")
    print(f"  ⬆️ Salaire max: {salary_data['salary_avg_k'].max():.1f}k€")

# Types de postes
print(f"\n🎯 Types de postes:")
print(f"  🔬 Data Scientist: {df_jobs['is_data_scientist'].sum()} ({df_jobs['is_data_scientist'].mean()*100:.1f}%)")
print(f"  📊 Data Analyst: {df_jobs['is_data_analyst'].sum()} ({df_jobs['is_data_analyst'].mean()*100:.1f}%)")
print(f"  ⚙️ Data Engineer: {df_jobs['is_data_engineer'].sum()} ({df_jobs['is_data_engineer'].mean()*100:.1f}%)")
print(f"  🎓 Alternance: {df_jobs['is_alternance'].sum()} ({df_jobs['is_alternance'].mean()*100:.1f}%)")

# Géographie
print(f"\n📍 Répartition géographique:")
print(f"  🏙️ Paris: {df_jobs['is_paris'].sum()} ({df_jobs['is_paris'].mean()*100:.1f}%)")
print(f"  🌍 Île-de-France: {df_jobs['is_ile_de_france'].sum()} ({df_jobs['is_ile_de_france'].mean()*100:.1f}%)")


🔍 Aperçu des premières lignes du DataFrame:


,id,title,description,created,contract_type,contract_time,salary_min,salary_max,salary_is_predicted,latitude,...,salary_max_k,salary_avg_k,title_clean,description_length,is_paris,is_ile_de_france,is_data_scientist,is_data_analyst,is_data_engineer,is_alternance
0,5234723007,Mission mentorat - Data Engineer,Description du poste Quelles sont les missions...,2025-06-05 18:08:34+00:00,contract,full_time,108000.0,192000.0,0,48.88140,...,192.0,150.0,Mission mentorat - Data Engineer,500,True,True,False,False,True,False
1,5222885856,Data scientist / data engineer,"Contexte Havea, leader européen de la santé na...",2025-05-30 06:12:58+00:00,None,None,NaN,NaN,0,NaN,...,NaN,NaN,Data scientist / data engineer,500,False,False,True,False,True,False
2,5234722960,Data Analyst - (H/F) - En alternance,Description du poste Vos missions en tant que ...,2025-06-05 18:08:33+00:00,contract,full_time,108000.0,192000.0,0,48.73030,...,192.0,150.0,Data Analyst - (H/F) - En alternance,500,False,True,False,True,False,True
3,5234723040,Data Analyst - (H/F) - En alternance,Description du poste Vos missions en tant que ...,2025-06-05 18:08:34+00:00,contract,full_time,108000.0,192000.0,0,48.87014,...,192.0,150.0,Data Analyst - (H/F) - En alternance,500,False,True,False,True,False,True
4,5234722999,Data Analyst - (H/F) - En alternance,Description du poste Vos missions en tant que ...,2025-06-05 18:08:34+00:00,contract,full_time,108000.0,192000.0,0,48.88140,...,192.0,150.0,Data Analyst - (H/F) - En alternance,500,True,True,False,True,False,True



📊 Informations générales:
  📋 Nombre d'offres: 35000
  📅 Période: 2017-10-20 à 2025-06-05
  🏢 Entreprises uniques: 1827
  📍 Lieux uniques: 487

💰 Informations salariales:
  📊 Offres avec salaire: 7945 (22.7%)
  💵 Salaire moyen: 36.8k€
  📈 Salaire médian: 42.5k€
  ⬇️ Salaire min: 0.0k€
  ⬆️ Salaire max: 299.0k€

🎯 Types de postes:
  🔬 Data Scientist: 2570 (7.3%)
  📊 Data Analyst: 10691 (30.5%)
  ⚙️ Data Engineer: 3795 (10.8%)
  🎓 Alternance: 4525 (12.9%)

📍 Répartition géographique:
  🏙️ Paris: 5232 (14.9%)
  🌍 Île-de-France: 18063 (51.6%)


## 📊 Analyses rapides

 `df_jobs` pour les analyses


In [11]:
# Exemples d'analyses :

# 1. Top 10 des entreprises qui recrutent
print("🏢 TOP 10 ENTREPRISES:")
top_companies = df_jobs['company_name'].value_counts().head(10)
for i, (company, count) in enumerate(top_companies.items(), 1):
    print(f"  {i:2d}. {company}: {count} offres")

# 2. Top 10 des localisations
print(f"\n📍 TOP 10 LOCALISATIONS:")
top_locations = df_jobs['location_display'].value_counts().head(10)
for i, (location, count) in enumerate(top_locations.items(), 1):
    print(f"  {i:2d}. {location}: {count} offres")

# 3. Analyse des salaires par type de poste
print(f"\n💰 SALAIRES PAR TYPE DE POSTE:")
for job_type, column in [('Data Scientist', 'is_data_scientist'), 
                        ('Data Analyst', 'is_data_analyst'), 
                        ('Data Engineer', 'is_data_engineer')]:
    subset = df_jobs[df_jobs[column] & df_jobs['salary_avg_k'].notna()]
    if len(subset) > 0:
        print(f"  🎯 {job_type}: {subset['salary_avg_k'].mean():.1f}k€ en moyenne ({len(subset)} offres)")

print(f"\n✅ Variable principale créée: df_jobs")
print(f"📊 Prêt pour vos analyses personnalisées !")


🏢 TOP 10 ENTREPRISES:
   1. Smile: 1384 offres
   2. Herbert Smith Freehills: 1202 offres
   3. Safran: 1073 offres
   4. Crédit Agricole: 1038 offres
   5. Techteam: 1028 offres
   6. Groupe SII: 788 offres
   7. Randstad: 780 offres
   8. Carrefour: 631 offres
   9. JEMS: 628 offres
  10. Sanofi Group: 619 offres

📍 TOP 10 LOCALISATIONS:
   1. France: 4627 offres
   2. Paris, Ile-de-France: 2421 offres
   3. Rhône, Auvergne-Rhône-Alpes: 1289 offres
   4. Bouches-du-Rhône, Provence-Alpes-Côte d'Azur: 1233 offres
   5. Massy, Palaiseau: 1064 offres
   6. Pont-Rousseau, Rezé: 1028 offres
   7. Rangueuil, Toulouse: 819 offres
   8. Seine-Saint-Denis, Ile-de-France: 790 offres
   9. Levallois-Perret, Nanterre: 674 offres
  10. Hauts-de-Seine, Ile-de-France: 666 offres

💰 SALAIRES PAR TYPE DE POSTE:
  🎯 Data Scientist: 27.2k€ en moyenne (122 offres)
  🎯 Data Analyst: 15.0k€ en moyenne (2238 offres)
  🎯 Data Engineer: 46.7k€ en moyenne (1630 offres)

✅ Variable principale créée: df_jobs
📊 P

## 💡 Variables disponibles pour vos analyses

**Variable principale : `df_jobs`** - DataFrame pandas avec toutes les offres d'emploi

### 📋 Colonnes principales :
- **Identifiants** : `id`, `title`, `company_name`
- **Dates** : `created` (datetime)
- **Salaires** : `salary_min_k`, `salary_max_k`, `salary_avg_k` (en k€)
- **Géographie** : `country`, `region`, `department`, `city`, `location_display`
- **Classification** : `is_paris`, `is_ile_de_france`, `is_data_scientist`, `is_data_analyst`, `is_data_engineer`, `is_alternance`
- **Contrat** : `contract_type`, `contract_time`
- **Autres** : `description`, `description_length`, `category_label`, `latitude`, `longitude`

### 🔍 Exemples d'analyses possibles :
```python
# Filtrer par type de poste
data_scientists = df_jobs[df_jobs['is_data_scientist']]

# Filtrer par localisation
jobs_paris = df_jobs[df_jobs['is_paris']]

# Filtrer par salaire
high_salary = df_jobs[df_jobs['salary_avg_k'] > 60]

# Grouper par entreprise
by_company = df_jobs.groupby('company_name').size().sort_values(ascending=False)
```

